In [4]:
def square_root_generator(limit): 
    n = 1 
    while n <= limit: 
        yield n ** 0.5 
        n += 1 

running_sum = 0
generator = square_root_generator(5) 
for sqrt_value in generator: 
    running_sum += sqrt_value
print(running_sum)

generator = square_root_generator(13) 
for sqrt_value in generator: 
    pass
print(sqrt_value)

8.382332347441762
3.605551275463989


In [1]:
%%capture
%pip install dlt[duckdb]
%pip install numpy
%pip install pandas

In [3]:
def people_1(): 
    for i in range(1, 6): 
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"} 
for person in people_1(): 
    print(person) 

def people_2(): 
    for i in range(3, 9): 
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"} 
for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [ ]:
import dlt
import duckdb

people_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

info = people_pipeline.run(people_1(), table_name='people', write_disposition='replace')
conn = duckdb.connect(f"{people_pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{people_pipeline.dataset_name}'")
display(conn.sql("SELECT sum(age) FROM people"))

info = people_pipeline.run(people_2(), table_name='people', write_disposition='append')
display(conn.sql("SELECT sum(age) FROM people"))

In [9]:
import dlt

people_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people2')
people_pipeline.run(people_1(), table_name='people2', write_disposition='replace', primary_key='ID')
people_pipeline.run(people_2(), table_name='people2', write_disposition='merge', primary_key='ID')

conn = duckdb.connect(f"{people_pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{people_pipeline.dataset_name}'")
print(conn.sql("SELECT sum(age) FROM people2"))

# print(conn.sql("SELECT * FROM people2 ORDER BY id"))

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707922178.9926457 │ WMMldwQFzmpXUA │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707922178.9926457 │ leo9fmKc2MTNbw │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1707922179.6415997 │ zwAJ/TW3qlJkAg │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1707922179.6415997 │ K+nAJgQitl64hQ │ Job_4      │
│     5 │ Person_5 │    35 │ City_B  │ 1707922179.6415997 │ mWWXkADPLaRlsQ │ Job_5      │
│     6 │ Person_6 │    36 │ City_B  │ 1707922179.6415997 │ 86oEo7TFD0onvw │ Job_6      │
│     7 │ Person_7 │ 